In [ ]:
import ...

In [ ]:
data = pd.read_csv(r'', sep = ',')

In [ ]:
data = data.query('flag_lk_nk == "LK"')

In [ ]:
data

In [ ]:
df=data.query('gs_with_disc > 0')

In [ ]:
df = df.drop(columns = ['status_mf', 'mb_trans','flag_lk_nk'])

In [ ]:
df = df.dropna()

In [ ]:
df.info()
df.isnull().sum()

In [ ]:
df['date'] = df['date'].astype('str')

In [ ]:
df['date'] = df['date'].map(lambda x: dt.datetime.strptime (x, '%Y%m%d'))

In [ ]:
df = df.query('gs_with_disc > 19')

In [ ]:
df['client_id'] = df['client_id'].astype('int64')
df['pcs'] = df['pcs'].astype('int64')
df['gs_with_disc'] = df['gs_with_disc'].astype('int64')
df['gs_without_disc'] = df['gs_without_disc'].astype('int64')
df['trans_numb'] = df['trans_numb'].astype('int64')

In [ ]:
df['gs_with_disc'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

In [ ]:
# здесь надо вписать нижний и верхний перцентиль
data_new = df.query('gs_with_disc > 99 and gs_with_disc < 5100 ')

In [ ]:
data_new['gs_with_disc'].describe(percentiles = [0.015,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.985]).apply("{0:.5f}".format)

In [ ]:
data_new = data_new.reset_index()

In [ ]:
data_new = data_new.drop(columns = 'index')

In [ ]:
data_new 

In [ ]:
gs_without_disc = data_new.groupby('client_id')['gs_with_disc','gs_without_disc','ns_with_disc', 'ns_without_disc','checks', 'pcs', 'cogs'].sum()
gs_without_disc = pd.DataFrame(gs_without_disc).reset_index()
gs_without_disc 

In [ ]:
lifetime_1 = data_new.groupby('client_id')['date'].max()
lifetime_1= pd.DataFrame(lifetime_1).reset_index()
lifetime_2 = data_new.groupby('client_id')['date'].min()
lifetime_2= pd.DataFrame(lifetime_2).reset_index()
lifetime = lifetime_1.merge(lifetime_2, left_on='client_id', right_on='client_id')
lifetime['lifetime'] = lifetime ['date_x'] - lifetime ['date_y']
lifetime = lifetime.sort_values(by = 'lifetime', ascending = False)
lifetime  = lifetime.rename(columns={"date_x": "last_pur_date", "date_y" : "first_pur_date"})
lifetime

In [ ]:
def recency(recency):

    res = str(recency).split(' ')[0]

    return(int(res))


lifetime['lifetime'] = lifetime['lifetime'].apply(recency)

In [ ]:
# сколько было покупок за год
amount_of_purch = data_new.groupby('client_id')['date'].count()
amount_of_purch= pd.DataFrame(amount_of_purch).reset_index()
amount_of_purch  = amount_of_purch.rename(columns={"date": "amount_purch"})
amount_of_purch

In [ ]:
rfm = amount_of_purch.merge(lifetime, how = 'left', left_on='client_id', right_on='client_id')
rfm = rfm.merge(gs_without_disc, how = 'left', left_on='client_id', right_on='client_id')
rfm

In [ ]:
rfm['UPT'] = rfm['pcs']/ rfm['checks']

In [ ]:
rfm['ASP']= rfm['ns_with_disc']/ rfm['pcs']

In [ ]:
rfm['dicount'] = rfm ['gs_without_disc'] - rfm ['gs_with_disc']

In [ ]:
rfm['dicount_pers'] = rfm['dicount']/rfm ['gs_without_disc']*100

In [ ]:
rfm['GPM'] = (rfm['ns_with_disc'] - rfm['cogs'])/rfm['ns_with_disc']

In [ ]:
rfm

In [ ]:
rfm =rfm.query('amount_purch < 10')
rfm

In [ ]:
rfm['recency'] = pd.to_datetime('2022-06-30') - rfm['last_pur_date']
rfm

In [ ]:
def recency(recency):

    res = str(recency).split(' ')[0]

    return(int(res))


rfm['recency'] = rfm['recency'].apply(recency)

In [ ]:
rfm['ATV'] = rfm['ns_with_disc']/rfm['amount_purch']

In [ ]:
rfm

In [ ]:
def frequency(rfm):
    frequency = []
    for i in range (len(rfm)):
        if rfm.iloc[i,1] == 1 :
            frequency.append(rfm.iloc[i,17]/rfm.iloc[i,1])
        else:
            frequency.append(rfm.iloc[i,4]/rfm.iloc[i,1])
    rfm['frequency'] = frequency      

In [ ]:
frequency(rfm)
rfm